In [1]:
path='/content/order.pdf'

In [2]:
### Libraries, put name in requirements.txt
!pip install tabula-py pandas sentence-transformers faiss-cpu google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 63.8 MB/s eta 0:00:00


In [3]:
## Importing Packages
import tabula
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from google.colab import userdata

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [4]:
## Load the Gem API Key
GEMINI_API_KEY = userdata.get('GEMINI_API')
if GEMINI_API_KEY is None:
  raise ValueError("Please set the GEMINI_API environment variable.")
genai.configure(api_key=GEMINI_API_KEY)

In [6]:
### Extracting the content
pdf_path='/content/order.pdf'
tables=tabula.read_pdf(pdf_path,pages='all')
df=pd.concat(tables,ignore_index=True)
print(df)

    order_id     order_status  ... order_quantity     sales
0          3  Order\rFinished  ...              6    523080
1        293  Order\rFinished  ...             49  20246040
2        483  Order\rFinished  ...             30   9931519
3        515  Order\rFinished  ...             19    788540
4        613  Order\rFinished  ...             12    187080
5        643  Order\rFinished  ...             21   5563640
6        678  Order\rReturned  ...             44    456820
7        807  Order\rFinished  ...             45    393700
8        868  Order\rFinished  ...             32   1433680
9        933  Order\rFinished  ...             15    161220
10       995  Order\rFinished  ...             46   3630980
11       998  Order\rFinished  ...             16    496520
12      1154  Order\rFinished  ...             44   8924460
13      1344  Order\rFinished  ...             15   1669808
14      1412  Order\rFinished  ...             13    118060
15      1539  Order\rFinished  ...      

In [7]:
df

,order_id,order_status,customer,order_date,order_quantity,sales
0,3,Order\rFinished,Muhammed Mac\rIntyre,13/10/2010,6,523080
1,293,Order\rFinished,Barry French,1/10/2012,49,20246040
2,483,Order\rFinished,Clay Rozendal,10/7/2011,30,9931519
3,515,Order\rFinished,Carlos Soltero,28/8/2010,19,788540
4,613,Order\rFinished,Carl Jackson,17/6/2011,12,187080
5,643,Order\rFinished,Monica Federle,24/3/2011,21,5563640
6,678,Order\rReturned,Dorothy Badders,26/2/2010,44,456820
7,807,Order\rFinished,Neola Schneider,23/11/2010,45,393700
8,868,Order\rFinished,Carlos Daly,8/6/2012,32,1433680
9,933,Order\rFinished,Claudia Miner,4/8/2012,15,161220


In [8]:
df.dtypes

,0
order_id,int64
order_status,object
customer,object
order_date,object
order_quantity,int64
sales,int64


In [10]:
# Storing Context
meta_data=df.describe(include='all').fillna("")
meta_data

,order_id,order_status,customer,order_date,order_quantity,sales
count,20.0,20,20,20,20.0,20.0
unique,,2,16,19,,
top,,Order\rFinished,Carlos Soltero,4/8/2012,,
freq,,19,3,2,,
mean,1003.65,,,,27.3,4032125.35
std,514.490272,,,,13.010522,6859259.145341
min,3.0,,,,6.0,118060.0
25%,635.5,,,,15.75,342045.0
50%,964.0,,,,26.5,764750.0
75%,1443.75,,,,35.75,4114145.0


In [14]:
## Converting df rows to text
# _ used for index
documents=[]
for _,row in df.iterrows():
  doc=(
      f"Order ID: {row['order_id']} has status {row['order_status']}\n"
      f"Customer is {row['customer']}."
      f"Order Date is {row['order_date']}."
      f"Quantity Ordered is {row['order_quantity']}."
      f"Total Sales amount is {row['sales']}."
  )
  documents.append(doc)
print(documents)

['Order ID: 3 has status Order\rFinished\nCustomer is Muhammed Mac\rIntyre.Order Date is 13/10/2010.Quantity Ordered is 6.Total Sales amount is 523080.', 'Order ID: 293 has status Order\rFinished\nCustomer is Barry French.Order Date is 1/10/2012.Quantity Ordered is 49.Total Sales amount is 20246040.', 'Order ID: 483 has status Order\rFinished\nCustomer is Clay Rozendal.Order Date is 10/7/2011.Quantity Ordered is 30.Total Sales amount is 9931519.', 'Order ID: 515 has status Order\rFinished\nCustomer is Carlos Soltero.Order Date is 28/8/2010.Quantity Ordered is 19.Total Sales amount is 788540.', 'Order ID: 613 has status Order\rFinished\nCustomer is Carl Jackson.Order Date is 17/6/2011.Quantity Ordered is 12.Total Sales amount is 187080.', 'Order ID: 643 has status Order\rFinished\nCustomer is Monica Federle.Order Date is 24/3/2011.Quantity Ordered is 21.Total Sales amount is 5563640.', 'Order ID: 678 has status Order\rReturned\nCustomer is Dorothy Badders.Order Date is 26/2/2010.Quantit

In [16]:
# Create Embeddings
embedding_model=SentenceTransformer('all-MiniLM-L6-v2')
embeddings=embedding_model.encode(documents,convert_to_numpy=True)

In [17]:
print(embeddings)

[[-0.04009407 -0.0381332   0.05240963 ... -0.08024696 -0.04358126
   0.05105153]
 [-0.03108117 -0.0188054  -0.00477832 ... -0.04830389 -0.05759399
  -0.01386822]
 [-0.06304456 -0.03100755  0.00952894 ... -0.03114312 -0.0744158
  -0.01718368]
 ...
 [ 0.02139982  0.00150404  0.07544634 ... -0.03270832 -0.01454125
  -0.02664857]
 [-0.09210813  0.01441542  0.04306164 ... -0.03566492 -0.00857274
   0.00213896]
 [-0.06764054 -0.00706391  0.00825971 ... -0.00477097 -0.08604836
   0.00569125]]


In [18]:
## Store embd in faiss
dimension=embeddings.shape[1]
index=faiss.IndexFlatL2(dimension)
index.add(embeddings)
faiss.write_index(index,'faiss_index.faiss') #saving the index file

In [20]:
### Retrieval function
def retrieve_context(query,k=3):
  query_embedding=embedding_model.encode([query])
  distances,indices=index.search(query_embedding,k)
  return "\n".join([documents[i] for i in indices[0]])

In [21]:
# Gemini Model configuration
# max_o/p_tokens, temp 0-1(helps create model response- creative(1) or deterministic(0))
generation_config={
    "temperature":0.4,
    "max_output_tokens":512,
}
print(generation_config)

{'temperature': 0.4, 'max_output_tokens': 512}


In [23]:
gemini_model=genai.GenerativeModel(model_name='models/gemini-2.5-flash',generation_config=generation_config)

In [28]:
# Conversational Bot
chat_history=[]
def chat_with_bot(user_input):
  global chat_history

  context=retrieve_context(user_input)
  prompt=f"""
  You are a helpful conversational data analyst assistant. Please refer to the context below and answer the user question.
  Context: {context}
  User Question: {user_input}

  Rules:
    - Be Conversational
    - Answer only using the context
    - If you don't know the answer, say you don't have enough information
  """
  response=gemini_model.generate_content(prompt)
  answer=response.text
  chat_history.append({"user":user_input, "bot":answer})
  return answer

In [29]:
## Final Step
print("Order Analytics Chat Bot Reasy !!!")
print("Type 'exit' to stop\n")
while True:
  user_input=input("User: ")
  if user_input.lower() in ["exit","quit","bye"]:
    print("Good Bye !!!")
    break
  response=chat_with_bot(user_input)
  print(f"Bot: ",{response})
  print("-"*60)

Order Analytics Chat Bot Reasy !!!
Type 'exit' to stop

User: what is the total sales?
Bot:  {'Based on the information provided, the total sales from all the orders listed is 2,528,828.'}
------------------------------------------------------------
User: who had highest sales?
Bot:  {'Based on the information provided, Jim Radford had the highest sales with a total of 1,669,808.'}
------------------------------------------------------------
User: exit
Good Bye !!!
